# Exercise 07

## Changes to the Evacuation Model

To enable exercises about learning and prediction, the evacuation model has been modified:

* Add statistics about the number of agents escaped through each of the exits (EscapedWest, EscapedSouth, EscapedEast, EscapedNorth)
* Introduce `turnwhenblocked_prop`: with this probability agents turn when their path is blocked by other agents
* If an agent is blocked, its `nervousness` is increased by `NERVOUSNESS_INCREASE_BLOCKED` (default: 0.2)
* When `random_spawn` is `False`, agents are located at the left part of the room.
* With low probability (`SPEED_RECOVERY_PROBABILTY = 0.15`), agents with `speed==0` increase their speed again (otherwise they likely become completely inactive)
* Add possibility to predict crowds while turning
* Introduce agent memory to store agents' cooperativeness and the number of steps it took the agent to escape. This allows the learning of favorable values of cooperativeness accross model runs.
* Introduce learning of `cooperativeness`
* Introduce `switches` for agents to switch on/off various features (eg. predictcrowd) - not implemented on model level to facilitate parameter passing)
* Double wall thickness from 1 to 2 to simulate severe bottlenecks.
* add `__repr__` and `__str__`to Floor object

## Evaluation Code


In [ ]:
from mesa.batchrunner import batch_run
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import sys
sys.path.insert(0,'../../abmodel')

from fire_evacuation.model import FireEvacuation
from fire_evacuation.agent import Human

unikcolors = [np.array((80,149,200))/255, np.array((74,172,150))/255,
                                                  np.array((234,195,114))/255, np.array((199,16,92))/255]
uniks = LinearSegmentedColormap.from_list( 'unik', unikcolors)

%matplotlib inline

In [ ]:
from app import page
page

In [ ]:
def run_model(model_a, model_b, maxsteps = 100):
    fig = plt.figure(figsize=(12, 12))
    model_a.run(maxsteps)
    model_b.run(maxsteps)
    
    ax = fig.add_subplot(2, 1, 2)
    ax.set_xlabel("Steps")
    ax.set_ylabel("Number of escaped through the exit")
    da = model_a.datacollector.get_model_vars_dataframe()[['EscapedWest', 'EscapedSouth','EscapedEast', 'EscapedNorth']]
    db = model_b.datacollector.get_model_vars_dataframe()[['EscapedWest', 'EscapedSouth','EscapedEast', 'EscapedNorth']]
    da.columns = ['EscapedWest_A', 'EscapedSouth_A','EscapedEast_A', 'EscapedNorth_A']
    db.columns = ['EscapedWest_B', 'EscapedSouth_B','EscapedEast_B', 'EscapedNorth_B']
    da.plot(ax=ax, colormap=uniks, ls="solid")
    db.plot(ax=ax, colormap=uniks, ls="dashed")
    
def plot_results(results):
    data = pd.DataFrame(results)[['predictcrowd', 'Step', 'seed','EscapedWest', 'EscapedSouth','EscapedEast', 'EscapedNorth']].round(decimals=1)

    data = data.groupby(data['Step'].diff().lt(1).cumsum()) \
        .apply(lambda x: x.set_index('Step').reindex(range(1, 101)).transform(lambda x: x.fillna(x.max()))) \
        .reset_index(level=0, drop=True) \
        .reset_index()

    fig, ax = plt.subplots(figsize=(12, 7))
    ax.set_ylabel("Escaped per exit")
    data[data['predictcrowd']==True].groupby(['Step']).agg("mean").drop(columns=['predictcrowd', 'seed']).plot(ax=ax, colormap=uniks, ls="solid")
    db = data[data['predictcrowd']==False].groupby(['Step']).agg("mean").drop(columns=['predictcrowd', 'seed'])
    db.columns = ['EscapedWest_NoPredict', 'EscapedSouth_NoPredict','EscapedEast_NoPredict', 'EscapedNorth_NoPredict']
    plot = db.plot(ax=ax, colormap=uniks, ls="dashed")

# Task 2 (Prediction in the evacuation model)

## Subtask 2.2

Currently, the time it takes to escape through a particular exit in the evacuation model (method `attempt_exit_plan()` in lines 387ff in `agent.py`) is predicted solely based on the distance from the current position to the exit door. How would you improve the prediction of time it takes to reach the exit? Try to sketch the required implementation in detail (<200 words). Provide pseudo code to illustrate your idea (learn how to write code in Markdown via *Help* > *Markdown reference*). Discuss your improvement in terms of cognitive load of a modelled actor. Implementing your proposal may help to find a suitable solution, but is no requirement. If you do so, consider implementing a switch to enable and disable your extention.

**Describe your ideas here**

## Subtask 2.3

When the switch `PREDICT_CROWD` is on, the turn() method (lines 346ff in agent.py) considers crowds such that the agent turns away from crowds. It therefore predicts the time it takes to escape from the number of agents in its field of vision (an agent does not consider that agents it perceives may head for different exits). Get a first impression whether this prediction helps by using the following code. Which three aspects can be obtained from the figure and should be considered for the analysis?

In [ ]:
# compare two single runs wrt. crowd prediction:
evacuation_a = FireEvacuation(floor_size = 14,
        human_count = 70,
        random_spawn = True,
        alarm_believers_prop = 0.7,
        max_speed = 2,
        cooperation_mean = 0.3,
        nervousness_mean = 0.5,
        predictcrowd = True,
        seed = 2)
evacuation_b = FireEvacuation(floor_size = 14,
        human_count = 70,
        random_spawn = True,
        alarm_believers_prop = 0.7,
        max_speed = 2,
        cooperation_mean = 0.3,
        nervousness_mean = 0.5,
        predictcrowd = False,
        seed = 2)

run_model(evacuation_a, evacuation_b, maxsteps = 100)

## Subtask 2.4

Comparing the feature of prediction for a single random seed is usually not sufficient. Specify the missing parameters and perform batch runs (50 seeds). Then, complement the function `analyse_significance_predictcrowd` to perfrom a t-test and visualise the distributions by boxplots. What is different when you set `random_spawn=False` (see description above)? Describe the results shortly, try to explain them and sketch a way to test your hypothesis (<200 words)!

In [ ]:
import numpy as np

params = dict(
    floor_size=14,
    human_count=125,
    alarm_believers_prop = 0.1,
    max_speed = 2,
    random_spawn = True,
    predictcrowd = # ?,
    cooperation_mean = 0.3,
    nervousness_mean = 0.5,
    seed = #? ,
)

results = batch_run(
        FireEvacuation,
        parameters=params,
        iterations = 1,
        max_steps = 200,
    )

In [ ]:
from scipy import stats

def analyse_significance_predictcrowd(data, title=""):

    print(f"Analysing {title}...")
    
    data = data.groupby(['predictcrowd', "seed"]).agg("mean").reset_index()
    # implement t-test

    # Interpret the results:
    # print appropriate messages about the t-test result, giving the level of significance
        
    data.boxplot(column=['Step'], by='predictcrowd', figsize = (12,6))
    None

In [ ]:
data = pd.DataFrame(results)
data = data[data['random_spawn']==True]
analyse_significance_predictcrowd(data, title="Random spawn agents")

In [ ]:
data = pd.DataFrame(results)
data = data[data['random_spawn']==False]
analyse_significance_predictcrowd(data, title="Clustered agents")

You may also want to analyse the escapes through the various exits:

In [ ]:
data = pd.DataFrame(results)
data = data[data['random_spawn']==False][['predictcrowd', 'Step', 'seed','EscapedWest', 'EscapedSouth','EscapedEast', 'EscapedNorth']]

# extend steps to max_steps grouped by seed and predictgrowd
data = data.groupby(data['Step'].diff().lt(1).cumsum()) \
    .apply(lambda x: x.set_index('Step').reindex(range(1, 201)).transform(lambda x: x.fillna(x.max()))) \
    .reset_index(level=0, drop=True) \
    .reset_index()

fig, ax = plt.subplots(figsize=(12, 7))
ax.set_ylabel("Escaped per exit")
data[data['predictcrowd']==True].groupby(['Step']).agg("mean").drop(columns=['predictcrowd', 'seed']).plot(ax=ax, colormap=uniks, ls="solid")
db = data[data['predictcrowd']==False].groupby(['Step']).agg("mean").drop(columns=['predictcrowd', 'seed'])
db.columns = ['EscapedWest_NoPredict', 'EscapedSouth_NoPredict','EscapedEast_NoPredict', 'EscapedNorth_NoPredict']
plot = db.plot(ax=ax, colormap=uniks, ls="dashed")

**Describe your finding here (learn about Markdown syntax via Menu > Help > Markdown Reference)!**

## Subtask 2.5

Find one other factor than crowd prediction (by variations in the according parameter) that has a significant impact on the evacutation time. Show appropriate plot(s) and explain the impact shortly (<100 words).

# Task 3 (Exploring learning in the evacuation model)

Agents can learn over time from past experience, and change their decision-making methods (the algorithms or perhaps only the parameters of those algorithms) as a consequence of their experience. When learning, agents seek to improve their behaviour according to an objective function.

In the evacuation model, the objective relates to the time it takes for an agent to escape the room. In this exercise, we're going to have the agents learn optimal values of cooperativeness. Therefore, agents memorise their cooperativeness and the time it takes them to leave the room. At the beginning, cooperation is changed randomly. After the memory is filled, agents identify the best evacuation (in terms of how long it takes until they escaped), choose the according cooperativeness value, and change it slightly to improve further:

## Subtask 3.1

Describe the meaning of the two parameter `COOPERATIVENESS_EXPLORATION` and `COOPERATIVENESS_CHANGE`. What makes learning difficult in the evacuation model? Discuss two reasons (think of determinism/path-dependency and coordination) (<200 words).

    COOPERATIVENESS_EXPLORATION = 0.0
    COOPERATIVENESS_CHANGE = 0.2
    
Execute the following code:

In [ ]:
from mesa.batchrunner import batch_run
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import sys
sys.path.insert(0,'../../abmodel')

from fire_evacuation.model import FireEvacuation
from fire_evacuation.agent import Human


unikcolors = [np.array((80,149,200))/255, np.array((74,172,150))/255,
                                                  np.array((234,195,114))/255, np.array((199,16,92))/255]
uniks = LinearSegmentedColormap.from_list( 'unik', unikcolors)

def run_model(model_a, model_b, maxsteps = 100):
    fig = plt.figure(figsize=(12, 12))
    model_a.run(maxsteps)
    model_b.run(maxsteps)
    
    ax = fig.add_subplot(2, 1, 2)
    ax.set_xlabel("Steps")
    ax.set_ylabel("Number of escaped through the exit")
    da = model_a.datacollector.get_model_vars_dataframe()[['EscapedWest', 'EscapedSouth','EscapedEast', 'EscapedNorth']]
    db = model_b.datacollector.get_model_vars_dataframe()[['EscapedWest', 'EscapedSouth','EscapedEast', 'EscapedNorth']]
    db.columns = ['EscapedWest_B', 'EscapedSouth_B','EscapedEast_B', 'EscapedNorth_B']
    da.plot(ax=ax, colormap=uniks, ls="solid")
    db.plot(ax=ax, colormap=uniks, ls="dashed")

When commenting in the line

`memories = memories[memories['agent'].values == observedAgent]`

learning only applies to `observedAgent`.

In [ ]:
from IPython.utils import io
observedAgent = 7

evacuation = FireEvacuation(floor_size = 14,
        human_count = 90,
        alarm_believers_prop = 0.7,
        max_speed = 2,
        cooperation_mean = 0.3,
        nervousness_mean = 0.5,
        predictcrowd = True,
        agentmemorysize = 7,
        seed = 3)

# Run the model
evacuation.run(100)

# Store the agent memory
memories = evacuation.get_agentmemories()
#Allow learning only for observed agent if commented in
#memories = memories[memories['agent'].values == observedAgent]

# Rerun the model starting with state of agents' memory of last run
for _ in range(0,20):
    evacuation = FireEvacuation(floor_size = 14,
            human_count = 70,
            alarm_believers_prop = 1.0,
            max_speed = 2,
            cooperation_mean = 0.3,
            nervousness_mean = 0.5,
            predictcrowd = True,
            agentmemories = memories,
            agentmemorysize = 7,
            seed = 3)
    evacuation.run(100)
    memories = evacuation.get_agentmemories()
    #Allow learning only for observed agent if commented in
    #memories = memories[memories['agent'].values == observedAgent]
    counter = 0
    steps2escape = 0
    cooperativeness = 0
    for agent in evacuation.schedule.agents:
        if isinstance(agent, Human):
            counter +=1
            steps2escape += agent.numsteps2escape
            cooperativeness += agent.cooperativeness

    print(f"Avg. steps to escape: {steps2escape/counter:.2f} | cooperativeness: {cooperativeness/counter:.2f}"
          f" | last step: {evacuation.schedule.steps}")

In [ ]:
# You may want to expore single agent's memory. Note that only the last *memorysize* entries are considered.
observedAgent = 3
memories = evacuation.get_agentmemories()
df = memories[memories['agent'].values==observedAgent]
df.sort_values('rep').set_index('rep').drop(columns=['cooperativeness','agent']).plot()
df.sort_values('cooperativeness').set_index('cooperativeness').drop(columns=['rep','agent']).plot()
df

In [ ]:
# You may want to expore single agent's memory. Note that only the last *memorysize* entries are considered.
observedAgent = 10
memories = evacuation.get_agentmemories()
df = memories[memories['agent'].values==observedAgent]
df.sort_values('rep').set_index('rep').drop(columns=['cooperativeness','agent']).plot()
df.sort_values('cooperativeness').set_index('cooperativeness').drop(columns=['rep','agent']).plot()
df

As you may observe, the implemented way of learning is not always successful. What makes learning difficult in the evacuation model? Discuss two reasons (think of determinism/path-dependency and coordination) (<200 words).

## Subtask 3.2

Find arguments pro and contra the statement that changing cooperativeness level in the evacuation model is learning (format as list)!